In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
import glob

def get_top_trending_stocks(num_stocks: int = 5) -> list:
    # find current path and print it
    print(os.getcwd())
    
    all_files = glob.glob('../data/*.csv')
    df_list = []

    for filename in all_files:
        symbol = os.path.basename(filename).split('.')[0]
        temp_df = pd.read_csv(filename, on_bad_lines='skip')
        temp_df['Symbol'] = symbol
        df_list.append(temp_df)

    df = pd.concat(df_list)
    df['Date'] = pd.to_datetime(df['Date'], format='mixed')

    # Convert relevant columns to numeric types
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

    # Calculate 5-day trends
    df['5_day_change'] = df.groupby('Symbol')['Close'].pct_change(5)

    # Get latest 5-day change for each stock
    latest_trends = df.groupby('Symbol')['5_day_change'].last().sort_values(ascending=False)

    # Print top trending stocks
    # print(f"Top {num_stocks} Trending Stocks:")
    # print(latest_trends.head(num_stocks))

    return latest_trends.head(num_stocks).index.tolist()


# Call the function and save results
top_trending_stocks = get_top_trending_stocks(num_stocks=10)
print(f"Top trending stocks: {top_trending_stocks}")


In [ ]:


def plot_top_trending_stocks(top_trending_stocks):
    for symbol in top_trending_stocks:
        filename = f'{symbol}.csv'
        stock_data = pd.read_csv(f'../data/{filename}', on_bad_lines='skip')
        
        # Skip the first 3 rows and reset the header
        stock_data = stock_data.iloc[3:].reset_index(drop=True)
        stock_data.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']

        # Print the columns to debug
        print(f"Columns in {filename}: {stock_data.columns}")

        if 'Date' in stock_data.columns and 'Close' in stock_data.columns:
            stock_data['Date'] = pd.to_datetime(stock_data['Date'], format='mixed')
            stock_data['Close'] = pd.to_numeric(stock_data['Close'], errors='coerce')

            # Calculate percentage increase
            stock_data['Pct_Change'] = stock_data['Close'].pct_change() * 100

            fig, ax1 = plt.subplots(figsize=(10, 5))

            ax1.plot(stock_data['Date'], stock_data['Close'], label=f'{symbol} Close Price', color='blue')
            ax1.set_xlabel('Date', fontweight='bold')
            ax1.set_ylabel('Close Price', color='blue')
            ax1.tick_params(axis='y', labelcolor='blue')
            ax1.legend(loc='upper left')
            ax1.grid(True)
            ax1.xaxis.set_major_locator(mdates.MonthLocator())
            ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
            plt.xticks(rotation=45, fontweight='bold')

            ax2 = ax1.twinx()
            ax2.plot(stock_data['Date'], stock_data['Pct_Change'], label=f'{symbol} % Change', color='orange')
            ax2.set_ylabel('Percentage Change', color='orange')
            ax2.tick_params(axis='y', labelcolor='orange')
            ax2.legend(loc='upper right')

            # Add y=0 line
            ax2.axhline(y=0, color='black', linewidth=2, linestyle='--')

            # Add big red star for volatile days
            volatile_days = stock_data[abs(stock_data['Pct_Change']) > 10]
            ax2.scatter(volatile_days['Date'], volatile_days['Pct_Change'], color='red', s=100, marker='*', label='Volatile Day')
            for idx, row in volatile_days.iterrows():
                ax2.text(row['Date'], row['Pct_Change'], 'Volatile', color='red', fontsize=12, fontweight='bold')

            plt.title(f'{symbol} Stock Price and Percentage Change')
            plt.tight_layout()
            plt.savefig(f'../data/{symbol}_trend_pct_change.png')
            plt.show()
        else:
            print(f"Skipping {symbol} as it does not contain 'Date' or 'Close' column")

# Call the function to plot the charts

# Call the function and save results
top_trending_stocks = get_top_trending_stocks(num_stocks=10)
# plot_top_trending_stocks(top_trending_stocks)


In [ ]:
from utils import CHART_TRADING_VIEW_TEMPLATE, display_chart_in_browser

In [ ]:


US_STOCKS = {
    "AZO": {
        "symbol": "AZO",
        "interval": "30",
        "Note": "uptrading",
        "layout": "VFWyLZpO"
    },
    "AZO_VIDYA": {
        "symbol": "AZO",
        "interval": "1H",
        "layout": "vZjXf8fw"
    },

    "BABA": {
        "symbol": "BABA",
        "interval": "5",
        "Note": "uptrading, Chinese new year",
        "layout": "VFWyLZpO"
    }
}

# Example usage
# display_charts_from_dict(US_STOCKS)


# Visual inspection

In [ ]:
from utils import get_top_trending_stocks, display_chart_in_browser, display_charts_from_dict

top_trending_stocks = get_top_trending_stocks(num_stocks=2)
print(f"Top trending stocks: {top_trending_stocks}")
# display_chart_in_browser(top_trending_stocks, url_template=CHART_TRADING_VIEW_TEMPLATE)

In [ ]:
import pprint
from utils import create_top_trending_stocks_dict

# Example usage
top_trending_stocks_dict = create_top_trending_stocks_dict(top_trending_stocks)
# get the top trending stocks



pprint.pprint(top_trending_stocks_dict)
# display first two stocks
# display_charts_from_dict(top_trending_stocks_dict)    

display_charts_from_dict(top_trending_stocks_dict)

# Closer reviews

In [ ]:
display_charts_from_dict(US_STOCKS)

In [ ]:
from utils import DAILY_BUYING_SCREEN_TEMPLATE, display_chart_in_browser

stock_symbols = ['UBER', 'YUM']
schwab_main = ['byddf', 'spy', 'mchi']

# display_chart_in_browser(stock_symbols, url_template=CHART_TRADING_VIEW_TEMPLATE)
# display_chart_in_browser(stock_symbols, url_template=DAILY_SCREEN_10_30_1H_TEMPLATE)
display_chart_in_browser(schwab_main, url_template=DAILY_BUYING_SCREEN_TEMPLATE)

In [ ]:
# Example usage
top_trending_stocks_dict = create_top_trending_stocks_dict(stock_symbols)
# get the top trending stocks



pprint.pprint(top_trending_stocks_dict)
# display first two stocks
# display_charts_from_dict(top_trending_stocks_dict)    

display_charts_from_dict(top_trending_stocks_dict)